In [ ]:
# %config InlineBackend.figure_format = 'retina'
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set()

# Homework 4

Austin Gill

* 5: 2, 10abd
* 9: 1, 2
* 10: 1, 2
* Text addition

## Problem 5.2

Using Veranda and the basic motion algorithm, place a set of obstacles that cause the robot to cycle and not find the goal. In other words, build a robot trap.

Excuse me whilst I vomit Python code all over. Here's the ROS node for the Basic Motion algorithm. Note that I stuggled finding a way to implement it based on the callbacks (can't put while loops in them, or you'll block everything and its mother) so I wrote the algorithm in a separate thread that can block all it wants without impacting the callbacks.

```python
import math
import threading as th
import time

import numpy as np
from geometry_msgs.msg import Pose2D
from rclpy.node import Node
from std_msgs.msg import ByteMultiArray, Float32


class BasicMotionController(Node):
    """A ROS node implementing the Basic Motion Algorithm.

    Algorithm:
        set heading towards goal
        while not arrived:
            while not obstructed:
                move forward
            count = 0
            while count < N:
                while obstacle in front:
                    turn right (or left)
                move forward
                count += 1
            set heading towards goal
    """

    @staticmethod
    def _pose_equal(pose1, pose2, tol=2.0, theta=False):
        """Compare two Pose2D poses for equality.

        :param pose1: The first Pose2D pose to compare.
        :param pose2: The second Pose2D pose to compare.
        :param tol: How close is close enough.
        :param theta: Whether or not to compare the pose headings.
        """
        p1 = [pose1.x, pose1.y]
        p2 = [pose2.x, pose2.y]
        if theta:
            p1.append(pose1.theta)
            p2.append(pose2.theta)

        p1 = np.array(p1)
        p2 = np.array(p2)

        distance = np.sqrt(np.sum((p2 - p1)**2))

        return distance <= tol

    @staticmethod
    def remap_angle(angle):
        """Remaps the given angle to the range [0, 2pi]"""
        return np.mod(angle, 2*np.pi)

    def __init__(self, position_topic, goal_topic, contact_topic, left_control_topic, right_control_topic):
        """Create a basic motion planner for controlling motion to goal.

        :param position_topic: The topic for the current position.
        :param goal_topic: The topic for the goal position.
        :param contact_topic: The topic for the contact sensor.
        :param left_control_topic: The topic for controlling the left wheel speed.
        :param right_control_topic: The topic for controlling the right wheel speed.
        """
        super().__init__('BasicMotionPlanner')

        self.base_speed = 1.0

        self.position = Pose2D()
        self.goal = Pose2D()
        # A ten sensor contact ring.
        self.contacts = [False] * 10
        # The maximum number of turns before attempting motion to goal again.
        self.max_iters = 5

        self.left = self.create_publisher(Float32, left_control_topic)
        self.right = self.create_publisher(Float32, right_control_topic)

        self.__pos_sub = self.create_subscription(Pose2D, position_topic, self.update_position)
        self.__goal_sub = self.create_subscription(Pose2D, goal_topic, self.update_goal)
        self.__contact_sub = self.create_subscription(ByteMultiArray, contact_topic, self.update_contacts)

        # BUG: This thread never dies...
        self.job = th.Thread(target=self.algorithm, daemon=True)
        self.job.start()

    def update_goal(self, pose):
        """Update the position of the goal."""
        self.goal = pose

    def update_position(self, pose):
        """Update the current robot position."""
        # Rotate so that 0rad is in pos x direction instead of pos y...
        # TODO: <rant></rant>
        pose.theta = self.remap_angle(pose.theta + np.pi/2)
        self.position = pose

    def update_contacts(self, msg):
        """Update the contact sensor array."""
        self.contacts = [bool(ord(b)) for b in msg.data]

    def algorithm(self):
        """Top level container for the Basic Motion Algorithm."""
        # TODO: Find a way to start() the thread once the node starts spinning.
        print('Waiting for node to spin.')
        while self.position == Pose2D() or self.goal == Pose2D():
            pass

        self.head_towards_goal()
        while not self.arrived():
            while not self.obstructed():
                self.move_forward()
            iters = 0
            while iters <= self.max_iters:
                while self.obstructed():
                    self.turn(self.remap_angle(self.position.theta - np.pi/4))
                self.move_forward()
                # Need to give it a chance to drive forward.
                time.sleep(0.1)
                iters += 1
            self.head_towards_goal()
        self.stop()

    def turn(self, heading):
        """Turn the robot in-place to the given heading.

        :param heading: The desired heading of the robot, in radians.

        Note that this function will block until the desired heading is achieved.
        """
        msg = Float32()
        self.stop()

        # BUG: There's a bug here, because the heading and pose orientation are
        # both remapped. But what happens when orientation is in Q1 and heading is in Q4?
        if heading - self.position.theta < 0:
            msg.data = self.base_speed
            self.left.publish(msg)
            msg.data = -self.base_speed
            self.right.publish(msg)
        else:
            msg.data = self.base_speed
            self.right.publish(msg)
            msg.data = -self.base_speed
            self.left.publish(msg)

        # Wait until the GPS heading roughly aligns with the desired heading
        while not math.isclose(self.position.theta, heading, abs_tol=0.2):
            pass

        self.stop()

    def arrived(self):
        """Determines whether the robot has arrived at its goal"""
        # Do not require orientations to be equal, because I'm not *that* masochistic.
        return self._pose_equal(self.position, self.goal, tol=2.0, theta=False)

    def move_forward(self):
        """Drives the robot forward in a straight line until directed otherwise.

        Nonblocking.
        """
        msg = Float32()
        msg.data = self.base_speed
        self.left.publish(msg)
        self.right.publish(msg)

    def head_towards_goal(self):
        """Turn the robot to face the goal, and drives forward.

        Blocks until the robot's heading is set, then sets the wheel speeds.
        """
        print('Heading towards goal.')
        # Get the vector from the robot to the goal.
        robot = np.array([self.position.x, self.position.y])
        goal = np.array([self.goal.x, self.goal.y])
        v = goal - robot
        # Compute the direction from the robot position to the goal.
        heading = self.remap_angle(np.arctan2(v[1], v[0]))
        print('Vector:', v)
        print('Heading:', heading)
        # Turn to that heading.
        self.turn(heading)
        # Set the wheel speeds and exit.
        self.move_forward()

    def obstructed(self):
        """Determine whether the robot is obstructed by an obstacle."""
        # Sensors 1, 2, and 3 are front right, front, and front left respectively.
        return any(self.contacts[1:4])

    def stop(self):
        """Stop the robot's motion"""
        msg = Float32()
        msg.data = 0.0
        self.left.publish(msg)
        self.right.publish(msg)
```

This node is used as follows

```python
#!/usr/bin/env python3
import sys
sys.path.append('../..')

import rclpy

from robotics.nodes import GpsPlotter, NodeManager, BasicMotionController


def main():
    """An example of live plotting a robot path in Veranda"""
    rclpy.init()
    manager = NodeManager(policy='multi')

    controller = BasicMotionController(position_topic='robot/gps',
                                       goal_topic='goal/gps',
                                       contact_topic='robot/contact',
                                       left_control_topic='robot/left_wheel',
                                       right_control_topic='robot/right_wheel')
    plotter = GpsPlotter('GpsPlotter', 'robot/gps')

    manager.add_node(controller)
    manager.add_node(plotter)

    manager.run()
    rclpy.shutdown()


if __name__ == '__main__':
    main()
```

See the included video file `basic_motion_5x.mp4` for the result.

## Problem 5.10.a

Implement Bug 1 in Veranda. Screen capture the results.

Here's some more code for your personal enjoyment.

```python
import math
import threading
import time
from collections import deque

import numpy as np
from geometry_msgs.msg import Pose2D, Twist
from rclpy.node import Node
from sensor_msgs.msg import LaserScan
from std_msgs.msg import ByteMultiArray


class BugOneController(Node):
    """A ROS node to Implement the Bug 1 motion algorithm.

    Algorithm:

    while True:
        repeat:
            from q_l,i-1 move to goal
        until goal is reached or obstacle encountered at q_h,i
        if goal is reached:
            exit
        repeat:
            follow boundary, recording path.
        until goal is reached or q_h,i is reencountered
        determine point q_l,i on path that is closest to goal
        go to q_l,i
        if robot cannot move towards goal:
            exit
    """

    @classmethod
    def _pose_equal(cls, pose1, pose2, tol=2.0, theta=False):
        """Compare two Pose2D poses for equality.

        :param pose1: The first Pose2D pose to compare.
        :param pose2: The second Pose2D pose to compare.
        :param tol: How close is close enough.
        :param theta: Whether or not to compare the pose headings.
        """
        return cls._pose_distance(pose1, pose2, theta) <= tol

    @staticmethod
    def _pose_distance(pose1, pose2, theta=False):
        """Find the Euclidean distance between two poses.

        :param pose1: The first pose.
        :type pose1: geometry_msgs.msg.Pose2D
        :param pose2: The second pose.
        :type pose2: geometry_msgs.msg.Pose2D
        :param theta: Whether to include the pose orientations, defaults to False.
        :type theta: bool, optional
        """
        p1 = [pose1.x, pose1.y]
        p2 = [pose2.x, pose2.y]
        if theta:
            p1.append(pose1.theta)
            p2.append(pose2.theta)

        p1 = np.array(p1)
        p2 = np.array(p2)

        return np.sqrt(np.sum((p2 - p1)**2))

    @staticmethod
    def remap_angle(angle):
        """Remap the given angle to the range [0, 2pi]."""
        return np.mod(angle, 2*np.pi)

    def __init__(self, gps_topic, lidar_topic, twist_topic, goal_topic, contact_topic, speed=2.0, target_distance=0.5, Kp=1.2, Ki=1.0, Kd=1.1, Kth=1.1):
        """Create a bug controller.

        :param gps_topic: The robot GPS sensor topic.
        :param lidar_topic: The robot LIDAR sensor topic.
        :param twist_topic: The robot control topic.
        :param goal_topic: The robot goal topic.
        :param contact_topic: The robot touch sensor topic.
        :param speed: The robot linear velocity.
        :param target_distance: The target distance to the boundary when wall-following.
        :param Kp: The proportional gain on the distance error term.
        :param Ki: The integral gain on the distance error term.
        :param Kd: The derivative gain on the distance error term.
        :param Kth: The proportional gain on the angular error term.
        """
        super().__init__('BugOneController')

        self.base_speed = speed
        # The angular velocity when turning in-place.
        self.turn_speed = speed / 3

        # Wall following configuration.
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.Kth = Kth
        self.target_distance = target_distance
        self.error = 0
        # Error calculation is at 10 Hz, so keep only a few seconds of history.
        self.errors = deque(maxlen=15)

        self.create_subscription(LaserScan, lidar_topic, self.update_lidar)
        self.create_subscription(Pose2D, gps_topic, self.update_position)
        self.create_subscription(Pose2D, goal_topic, self.update_goal)
        self.create_subscription(ByteMultiArray, contact_topic, self.update_contact)

        self.pub = self.create_publisher(Twist, twist_topic)

        # TODO: I should technically use locks, but it hasn't become an issue yet.
        # The current state of the sensors
        self.position = Pose2D()
        self.goal = Pose2D()
        self.contacts = [False] * 10

        # The q_l and q_h points
        self.hit_point = Pose2D()
        self.leave_point = Pose2D()

        self.boundary_follow = False

        self.job = threading.Thread(target=self.algorithm, daemon=True)
        self.job.start()

    def update_lidar(self, scan):
        """Receive updates from the robot LIDAR sensor.

        :param scan: The LIDAR sweep
        :type scan: sensor_msgs.msg.LaserScan
        """
        if self.boundary_follow:
            l = len(scan.ranges)
            # The start and stop indices in the sweep for the left side.
            i_0 = l // 2
            i_n = l

            min_distance = scan.ranges[i_0]
            min_i = i_0

            for i, d in enumerate(scan.ranges[i_0:i_n + 1]):
                if d < min_distance:
                    min_distance = d
                    min_i = i

            if min_distance > scan.range_max:
                print('Infinite range on left side.')
                return

            # Get the min distance by hand so that we can also get the angle it occurs at.
            min_angle = (min_i - l // 2) * scan.angle_increment
            front_distance = scan.ranges[l // 2]

            # A poor man's derivative.
            error_d = min_distance - self.target_distance - self.error
            self.error = min_distance - self.target_distance
            # Note that errors has forgetfulness.
            self.errors.append(self.error)
            # Not really a true integral, but has a similar effect.
            error_i = sum(self.errors) / len(self.errors)

            self.controller(front_distance, min_angle, self.error, error_i, error_d)

    def controller(self, front_distance, min_angle, error, error_i, error_d):
        """PD controller to keep the robot at a set distance and angle from the boundary.

        :param front_distance: The distance reading in front of the robot.
        :param min_angle: The angle to the minimum distance.
        :param error: The error to correct.
        :param error_i: The integral of the error.
        :param error_d: The time derivative of the error.
        """
        msg = Twist()
        # A hybrid PD controller to control both the distance and the heading of the robot.
        msg.angular.z = self.Kp * error +  self.Ki * error_i + self.Kd * error_d + self.Kth * (min_angle - np.pi / 2)

        if front_distance < self.target_distance:
            msg.linear.x = 0.0
        elif front_distance < 2 * self.target_distance:
            msg.linear.x = self.base_speed / 2
        elif abs(min_angle) > 1.75:
            msg.linear.x = self.base_speed / 2
        else:
            msg.linear.x = self.base_speed

        self.pub.publish(msg)

    def update_position(self, pose):
        """Receive update from the robot GPS sensor.

        :param pose: The GPS sensor reading
        :type pose: geometry_msgs.msg.Pose2D
        """
        # Rotate so that 0rad is in pos x direction instead of pos y...
        # TODO: <rant></rant>
        pose.theta = self.remap_angle(pose.theta + np.pi/2)
        self.position = pose

        if self.boundary_follow:
            if self._pose_distance(pose, self.goal) < self._pose_distance(self.leave_point, self.goal):
                self.leave_point = pose

    def update_goal(self, pose):
        """Receive updates from the robot's goal.

        :param pose: The goal GPS sensor reading
        :type pose: geometry_msgs.msg.Pose2D
        """
        self.goal = pose

    def update_contact(self, msg):
        """Receive updates from the robot's touch sensor.

        :param msg: The touch sensor reading.
        :type msg: std_msgs.msg.ByteMultiArray
        """
        # Convert the array of bytes to a list of bools. No need for struct.unpack!
        self.contacts = [bool(ord(b)) for b in msg.data]

    def algorithm(self):
        """Run the Bug 1 algorithm.

        A blocking function that runs in a separate thread to prevent blocking
        the ROS callbacks. This way the sensor readings are still updated and we
        have access to up-to-date information.
        """
        # TODO: Find a way to start() the thread once the node starts spinning.
        print('Waiting for node to spin.')
        while self.position == Pose2D() or self.goal == Pose2D():
            pass

        # The Bug 1 algorithm
        while True:
            print('Heading towards goal.')
            self.head_towards_goal()
            while not self.arrived() and not self.obstructed():
                self.move_forward()

            if self.arrived():
                break

            # Bring the robot to a stop, and back up slightly, and stop again.
            msg = Twist()
            self.pub.publish(msg)
            msg.linear.x = -self.base_speed
            msg.angular.z = -self.turn_speed
            self.pub.publish(msg)
            time.sleep(0.4)
            msg.linear.x = 0.0
            self.pub.publish(msg)

            # The hit point is the robot's current position.
            self.hit_point = self.position
            print(f'Hit point: {self.hit_point}')

            print('Starting boundary following.')
            self.follow_boundary()

            print(f'Leave point: {self.leave_point}')

            print('Heading towards leave point.')
            self.go_to_leave()

        # Stop the robot.
        msg = Twist()
        self.pub.publish(msg)
        print('Finished. Phagocytosis.')

    def follow_boundary(self):
        """Begin following an obstacle boundary."""
        self.boundary_follow = True
        print('Boundary following: Leaving hit point.')
        # Follow the boundary until we leave the hit point.
        while self.reencountered_last():
            pass

        print('Boundary following: Searching for hit point.')
        # Follow the boundary until we reencounter the hit point.
        while not self.arrived() and not self.reencountered_last():
            pass

        print(f'Position "at" hit point: {self.position}')

        self.boundary_follow = False

    def arrived(self):
        """Determine whether the robot has arrived at its goal."""
        # Do not require orientations to be equal, because I'm not *that* masochistic.
        # Also, it's getting late and I'm nowhere near done.
        return self._pose_equal(self.position, self.goal, tol=3.0, theta=False)

    def move_forward(self):
        """Drive the robot forward until directed otherwise."""
        msg = Twist()
        msg.linear.x = self.base_speed
        self.pub.publish(msg)

    def turn(self, heading):
        """Turn the robot in-place to face the given heading.

        Note that this function blocks until the robot's heading is correctly set.

        :param heading: The desired heading in radians.
        """
        msg = Twist()

        if heading - self.position.theta < 0:
            msg.angular.z = -self.turn_speed
        else:
            msg.angular.z = self.turn_speed

        self.pub.publish(msg)

        while not math.isclose(self.position.theta, heading, abs_tol=0.1):
            pass

        msg.angular.z = 0.0
        self.pub.publish(msg)

    def head_towards_goal(self):
        """Turn the robot to face the goal, then drive forward.

        Note that this function blocks until the robot's heading is correctly set.
        """
        # Get the vector from the robot to the goal.
        robot = np.array([self.position.x, self.position.y])
        goal = np.array([self.goal.x, self.goal.y])
        v = goal - robot
        # Compute the direction from the robot position to the goal.
        heading = self.remap_angle(np.arctan2(v[1], v[0]))

        self.turn(heading)
        self.move_forward()

    def obstructed(self):
        """Determine whether the robot is obstructed by an obstacle."""
        # Sensors 1, 2, and 3 are front right, front, and front left respectively.
        return any(self.contacts)

    def reencountered_last(self):
        """Determine if the robot has reencountered the hit point."""
        return self._pose_equal(self.position, self.hit_point, tol=0.8, theta=False)

    def go_to_leave(self):
        """Follow the obstacle wall until the leave point in encountered."""
        print('Following boundary to leave point.')
        self.boundary_follow = True
        while not self._pose_equal(self.position, self.leave_point, tol=0.5, theta=False):
            pass
        print(f'Position "at" leave point: {self.position}')
        self.boundary_follow = False
```

There are several interesting things in this implementation to take note of.

1. This node does not control wheel speeds directly. Instead, it publishes `Twist` messages with the robots linear and angular velocities in the robot's local reference frame.
2. Again, it runs the algorithm in a separate thread to avoid blocking the callbacks. I'd be quite interested to know what the "correct" solution to this would be.
3. It performs wall-following with a PID controller.
4. The description of the Bug 1, 2, and 3 algorithms states that they use contact sensors, but it was far simpler to implement wall following with a short-range LIDAR, since I had already implemented wall-following for my text addition.

The above node publishes `Twist` messages to the following node to set the wheel speeds.

```python
from geometry_msgs.msg import Twist
from rclpy.node import Node
from std_msgs.msg import Float32


class DdController(Node):
    """A ROS node to translate Twist messages to differential drive robot
    wheel speeds.
    """

    def __init__(self, twist_topic, left_topic, right_topic, r=0.5, L=0.6):
        """Constructs a DdController node.

        :param twist_topic: The topic publishing the Twist messages.
        :param left_topic: The topic to publish the left wheel speeds to.
        :param right_topic: The topic to publish the right wheel speeds to.
        :param r: The robot wheel radius.
        :param L: The robot half-axle length.
        """
        super().__init__('DdController')

        self.r = r
        self.L = L

        self.create_subscription(Twist, twist_topic, self.callback)
        self.left = self.create_publisher(Float32, left_topic)
        self.right = self.create_publisher(Float32, right_topic)

    def callback(self, msg):
        """Converts Twist messages with x and theta velocities (in local coordinates)
        to (left, right) wheel speeds.
        """
        left, right = Float32(), Float32()
        difference = msg.angular.z * 2 * self.L / self.r

        left.data = msg.linear.x
        right.data = msg.linear.x + difference

        self.left.publish(left)
        self.right.publish(right)
```

Then the two are used in conjunction as follows. Note that the gains were experimentally determined over a span of far too many hours.

```python
#!/usr/bin/env python3
import sys
sys.path.append('../..')

import rclpy

from robotics.nodes import GpsPlotter, NodeManager, BugOneController, DdController


def main():
    """Bug 1 motion algorithm demo."""
    rclpy.init()
    manager = NodeManager(policy='multi')

    # The Bug 1 controller sends twist messages to this controller.
    controller = DdController(twist_topic='robot/control/twist',
                              left_topic='robot/left_wheel',
                              right_topic='robot/right_wheel',
                             )

    bug_controller = BugOneController(gps_topic='robot/gps',
                                      lidar_topic='robot/lidar',
                                      twist_topic='robot/control/twist',
                                      goal_topic='goal/gps',
                                      contact_topic='robot/contact',
                                      speed=3.3,
                                      target_distance=0.5,
                                      Kp=3.0,
                                      Ki=3.5,
                                      Kd=3.0,
                                      Kth=3.1,
                                     )

    plotter = GpsPlotter('GpsPlotter', 'robot/gps', history=None)

    manager.add_node(plotter)
    manager.add_node(controller)
    manager.add_node(bug_controller)

    manager.run()
    rclpy.shutdown()


if __name__ == '__main__':
    main()
```

For your personal enjoyment, here are some failed attempts. This is a poorly tuned PID controller. I which I had pressed "save" on the GPS plot, but all I have is a screenshot :(

![Curly PID](figures/curly_pid.png)

I don't even know what this is.

![Broken Bug 1](figures/broken_bug_1.svg)

Here's what happens when you don't have your exit conditions set correctly. Your robot will just push the goal around the simulation!

![Exit Condition](figures/kinda_bug_1_work.svg)

Then finally, here's the path of a correctly tuned Bug.

![Bug 1 working](figures/bug1b.svg)

See the included `bug1b_5x.mp4` file for a video of the Bug 1 algorithm at work.

## Problem 9.1

Assume that you are working in a large event center which has beacons located around the facility. Estimate the location of a robot, $(a,b,c)$, if the $(x,y,z) $ location of the beacon and the distance from the beacon to the robot, $d$, are given in the table below.

| $x$ | $y$ | $z$ | $d$ |
|-----|-----|-----|-----|
| 884 | 554 | 713 | 222 |
| 120 | 703 | 771 | 843 |
| 938 | 871 | 583 | 436 |
| 967 | 653 | 46  | 529 |
| 593 | 186 | 989 | 610 |

First, we plot the spheres to get a good idea of what we're looking at.

In [ ]:
centers = [
    (884, 554, 713),
    (120, 703, 771),
    (938, 871, 583),
    (967, 653, 46),
    (593, 186, 989),
]

radii = [222, 843, 436, 529, 610]

spheres = list(zip(centers, radii))

In [ ]:
u = np.linspace(0, 2 * np.pi, 20)
v = np.linspace(0, np.pi, 20)

circles = []

# Generate the points on the spheres
for center, r in spheres:
    x = center[0] + r * np.outer(np.cos(u), np.sin(v))
    y = center[1] + r * np.outer(np.sin(u), np.sin(v))
    z = center[2] + r * np.outer(np.ones(np.size(u)), np.cos(v))
    circles.append((x, y, z))

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')
ax.view_init(azim=-10, elev=15)

for x, y, z in circles:
    ax.plot_surface(x, y, z, linewidth=0.05, alpha=0.6)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

# TODO: <rant></rant>
# ax.set_xlim(0, 1000) and ax.set_xbound(0, 1000) set the *axis* bounds
# not the data bounds, so the spheres still get plotted outside the axis...
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)
ax.set_zlim(0, 1000)

plt.show()

Now how to find the intersection point? For simplicity's sake, I will use the optimization technique given as the first example in the textbook. For this, we build an error function

$$f(x, y, z) = \sum_i \left( \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - r_i \right)^2$$

and find the point $(x, y, z)$ that minimizes the error and claim that that point is as close to an intersection of the spheres as we're going to get.

I will modify this error function to also take in a vector $\vec s$ of sphere indices to use. This way I can loop through all combinations of 3, 4, and 5 of the given spheres.

In [ ]:
def E(v, s):
    """Error function for the given set of spheres."""
    x, y, z = v
    e = 0
    for sphere in (spheres[i] for i in s):
        (xi, yi, zi), ri = sphere
        e += (np.sqrt((x - xi)**2 + (y - yi)**2 + (z - zi)**2) - ri)**2

    return e

In [ ]:
# A generator of all the sphere index vectors
ss = itertools.chain.from_iterable(itertools.combinations(range(5), i) for i in [3, 4, 5])

In [ ]:
results = []
for s in ss:
    results.append(sp.optimize.minimize(E, x0=(800, 400, 400), args=(s,)))

Note that the `minimize` function can fail, in which case the returned `OptimizeResult` object has the `success` field set to `False`.

In [ ]:
# Lists of (x, y, z) tuples
positions = np.array([r.x for r in results if r.success])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Unzip the list of tuples into a tuple of lists
ax.scatter(*zip(*positions), alpha=0.7)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.title('Optimization results for all combinations of 3, 4, and 5 spheres')

plt.show()

This gives us a bunch of potential intersection points. Yay. So we remove the two outliers and plot the centroid.

In [ ]:
x, y, z = zip(*positions)

filtered = []

# Filter out the outliers
for xi, yi, zi in zip(x, y, z):
    if xi > 860:
        filtered.append((xi, yi, zi))

filtered = np.array(filtered)

centroid = np.mean(filtered, axis=0)
print(f'centroid: {centroid}')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Unzip the list of tuples into a tuple of lists
ax.scatter(*zip(*filtered), alpha=0.7, label='Intersections')
ax.scatter(*centroid, label='Centroid')

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.title('Optimization results with outliers removed')
plt.legend()

plt.show()

So the estimated robot position is near $(886.03, 442.21, 521.77)$. If matplotlib would respect the `ax.set_*lim()` function calls when I plot the sphere surfaces, I could generate a helpful plot showing the spheres and the estimated position. But I've spent far too much of my life going through the first ten pages of my Google searches (I even used Google over DuckDuckGo for this) for me to try any more.

## Problem 9.2

If you are using a laser diode to build a distance sensor, you need some method to determine the travel time. Instead of using pulses and a clock, try using phase shifts. What is the wavelength of the modulated frequency of 10MHz? If you measure a 10 degree phase shift, this value corresponds to what distances? What if the phase shift measurement has noise: zero mean with standard deviation 0.1? How does one get a good estimate of position if the ranges to be measured are from 20 meters to 250 meters?

We have wavelength $\lambda = \frac{c}{f}$, where $c = 3 \times 10^8$ and $f = 10\times10^6$, so then $\lambda = 30$m.

A $10^\circ$ phase shift is $\frac{10}{360} = \frac{1}{36}$ of the wavelength, so with a wave length of $30$m, we have an estimate of the distance travelled of $30\cdot\frac{1}{36} = 0.8\overline{33}$ meters for the round trip distance.

Now, of course, realise that the phase shift would be the same for any round trip distance $0.8\overline{33} + 30n$ where $n \in \mathbb Z$.

If the phase shift reading is additively normally noisy, then the above value of $\frac{1}{36}$ will be replaced by some distribution with a mean and standard deviation. So we'd have instead, $X + 30n$, where $X$ is random. However, the size difference of $X$ relative to the size of $30n$ for almost any $n$ is small! So for small amounts of noise, our overall distance estimate (for this particular problem) is not severly impacted.

If we know something about our arena, we can place limits on the $n$ from above. Namely, $n \in \{1, \dots, 8\}$. However, this still gives too much uncertainty. So we should modulate the laser at two *relatively prime* frequencies so phase shifts will only coincide at a single unique distance.

Pick $17$MHz, which gives $\lambda = \frac{c}{f} = 17.6471$. Then $D_2 = \lambda \frac{\theta_2}{360} + m \lambda$. Then we find $n, m \in \mathbb Z$ for which $\lambda \frac{\theta_1}{360} + n \lambda = \lambda \frac{\theta_2}{360} + m \lambda$. Note, however, that this requires a second phase shift measurement $\theta_2$, which may have additional noise.

In a noisy system, we may (will) not have an exact equality between $D_1$ and $D_2$, so we'd have to find values of $n$ and $m$ for which they agree within some tolerance. We'd have to determine the tolerance based on the size of our arena (the scale at which we're working at) and the amount of noise we believe we see.

## Problem 10.1

Assume you have a laser triangulation system as shown in Fig. 10.2 given by (10.1) and that $f=8$mm, $b=30$cm. What are the ranges for $\alpha$ and $u$ if we need to measure target distances in a region (in cm) $20<z<100$ and $10<x<30$?

$$x = \frac{b u}{f\cot \alpha + u},  \quad
  z = \frac{b f}{f\cot \alpha + u}$$
  
![Fig 10.2](figures/lasertriangulation2.svg)

This is an exercise in $\LaTeX$ (a.k.a., algebra).

We have

$$\begin{cases}
x &= \frac{bu}{f \cot \alpha + u} \\
z &= \frac{bf}{f \cot \alpha + u}
\end{cases}$$

Solve this system of equations for $u$ and $\alpha$, then find the minimum and maximum values for $u$ and $\alpha$ as $x$ and $z$ range over the given domain.

Starting with $z$, we have

$$\begin{align*}
z &= \frac{bf}{f \cot \alpha + u} \\
z(f \cot \alpha + u) &= bf \\
zu &= bf - zf \cot \alpha \\
u &= \frac{bf}{z} - f \cot \alpha
\end{align*}$$

Then substitute $u = \frac{bf}{z} - f \cot \alpha$ into

$$\require{cancel}\begin{align*}
x &= \frac{bu}{f \cot \alpha + u} \\
  &= \frac{b\left(\frac{bf}{z} - f \cot \alpha\right)}{f \cot \alpha + \frac{bf}{z} - f \cot \alpha} \\
  &= \frac{\bcancel{f}b \left(\frac{b}{z} - \cot \alpha\right)}{\bcancel{f}\left(\cancel{\cot \alpha} + \frac{b}{z} - \cancel{\cot \alpha}\right)} \\
  &= \frac{b \left(\frac{b}{z} - \cot \alpha\right)}{\left(\frac{b}{z}\right)} \\
  &= \frac{\frac{b^2}{z} - b \cot \alpha}{\frac{b}{z}} \\
  &= \frac{z}{b}\left(\frac{b^2}{z} - b \cot \alpha\right) \\
  &= b - z \cot \alpha \\
z \cot \alpha &= b - x \\
\cot \alpha &= \frac{b - x}{z} \\
\alpha &= \cot^{-1}\left(\frac{b - x}{z}\right)
\end{align*}$$

So barring any algebra mistakes, we have the new (old, with new look) system of equations

$$\begin{cases}
u &= \frac{fx}{z} \\
\alpha &= \cot^{-1}\left(\frac{b - x}{z}\right)
\end{cases}$$

Now we can range over $10 < x < 30$ and $20 < z < 100$ to find the minimum and maximum values of $u$ and $\alpha$.

Since $b = 30$cm and $f = 0.8$cm, our system of equations only has two unknowns. Further, each component is relatively simple, so the minimum and maximum values of $u$ and $\alpha$ can be immediately picked out.

Since $x$ and $z$ are both positive, $u$ will have its minimum value at $(x, z) = (10, 100)$ of $\frac{0.8 \cdot 10}{100} = 0.08$ and its maximum value at $(x, z) = (30, 20)$ of $\frac{0.8 \cdot 30}{20} = 1.2$.

Similarly, since $\cot^{-1} x$ is strictly positive and monotonically decreasing over $x \in[0, \infty)$, $\alpha$ is greatest as the smallest value of $\frac{b - x}{z}$ and least at the largest value of $\frac{b - x}{z}$. So the largest value of $\alpha$ is $\cot^{-1}(0) = \frac{\pi}{2}$, and the smallest value of $\alpha$ is $\cot^{-1}\left(\frac{30 - 10}{20}\right) = \cot^{1} = \frac{\pi}{4}$.

So in summary, we found

$$\begin{cases}
u &= \frac{fx}{z} \\
\alpha &= \cot^{-1}\left(\frac{b - x}{z}\right)
\end{cases}$$

where $0.08 < u < 1.2$ and $\frac{\pi}{4} < \alpha < \frac{\pi}{2}$.

## Problem 10.2

Assume you have two cameras that are calibrated into a stereo pair with a baseline of $10$cm, and focal depth of $7$mm. If the error is $10\%$ on $v_1$ and $v_2$, $v_1=2$mm and $v_2=3$mm, what is the error on the depth measurement $z$? Your answer should be a percentage relative to the error free number. Hint: If $v_1=2$ then a $10\%$ error ranges from $1.8$ to $2.2$. (Although not required, another way to approach this problem is the total differential from calculus.)

Similar to problem 10.1, let $1.8 < v_1 < 2.2$ and $2.7 < v_2 < 3.3$ and determine the range of values $x$ and $z$ take.

$$\begin{cases}
x &= \frac{v_1 b}{v_1 + v_2} \\
z &= \frac{fb}{v_1 + v_2}
\end{cases}$$

Because the denominator of each of these fractions contains *both* $v_1$ and $v_2$, we can expect some curvature in the plotted surfaces. This means that we cannot just plug in the boundary of the domain and expect to find the extrema. However, if we look at the plot of $x(v_1, v_2)$ and $z(v_1, v_2)$ it appears we get lucky and can easily pick out the extrema. Wolfram Alpha agrees.

In [ ]:
# Change units from mm to cm.
v1 = np.linspace(0.18, 0.22, 10)
v2 = np.linspace(0.27, 0.33, 10)

v1, v2 = np.meshgrid(v1, v2)

x = v1 * 10 / (v1 + v2)
z = 0.7 * 10 / (v1 + v2)

In [ ]:
print(f'min x = {x.min()}, max x = {x.max()}')
print(f'min z = {z.min()}, max z = {z.max()}')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(azim=45, elev=30)

plt.title('$x$ as a function of $v_1$ and $v_2$')
ax.plot_surface(v1, v2, x, linewidth=0, alpha=0.8)
# ax.plot_surface(v1, v2, x.min() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')
# ax.plot_surface(v1, v2, x.max() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')

ax.set_xlabel('$v_1$')
ax.set_ylabel('$v_2$')
ax.set_zlabel('$x$')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
# ax.view_init(azim=45, elev=30)

plt.title('$z$ as a function of $v_1$ and $v_2$')
ax.plot_surface(v1, v2, z, linewidth=0, alpha=0.8)
# ax.plot_surface(v1, v2, z.min() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')
# ax.plot_surface(v1, v2, z.max() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')

ax.set_xlabel('$v_1$')
ax.set_ylabel('$v_2$')
ax.set_zlabel('$z$')

plt.show()

In [ ]:
print(12.727 / 14)
print(15.556 / 14)

The minimum of $z(v_1, v_2)$ can be found at $(v_1, v_2) = (0.22, 0.33)$ with value $z = 12.\overline{72}$. The maximum occurs at $(v_1, v_2) = (0.18, 0.27)$ with value $z = 15.\overline{55}$.

So $z$ has range $12.727 < z < 15.556$ with $10\%$ error on both $v_1$ and $v_2$. The actual value is $z = 14$cm.

Since percentages are hard, I always have to do

$$14 p_l = 12.727$$

and

$$14 p_h = 15.556$$

to get proportions $p_l = 0.909$ and $p_h = 1.111$. In the region close to the actual value of $z = 14$cm, we can assume linearity, and claim that the error in the measured $z$ value versus the actual value is $\sim 10\%$. But note that the error is will grow as $v_1$ and $v_2$ decrease due to the curvature of the surface, which becomes dramatic for larger ranges of $v_1$ and $v_2$.